# In my project, I am using a Qwiklabs account instead of a Google cloud account to use features on GCP :))

## Setting Path

In [2]:
# Use the latest version of pip.
!pip install -U --upgrade pip
!pip install -U --upgrade "tfx[kfp]<2"

  Obtaining dependency information for tfx[kfp]<2 from https://files.pythonhosted.org/packages/3c/e1/919574a8d34382c09ce0c74398dfa615d368d210741773263fa716ffcda6/tfx-1.14.0-py3-none-any.whl.metadata
  Using cached tfx-1.14.0-py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for ml-pipelines-sdk==1.14.0 from https://files.pythonhosted.org/packages/45/90/8b25072e398f8772521374c5d4315a74cb7bfd30aaa7ab3e399d95d4faa4/ml_pipelines_sdk-1.14.0-py3-none-any.whl.metadata
  Using cached ml_pipelines_sdk-1.14.0-py3-none-any.whl.metadata (2.3 kB)
  Obtaining dependency information for apache-beam[gcp]<3,>=2.47 from https://files.pythonhosted.org/packages/2e/7c/43adabfa0e14014419cd9baa52e8923f524d52349239bebde6e0f3d89248/apache_beam-2.50.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached apache_beam-2.50.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
  Using cached google_cloud_bigquery-2.34.4-py2.py3-none-any.whl (

In [2]:
import tensorflow_model_analysis as tfma

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

2023-09-18 02:44:30.712256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.13.0


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIF

TFX version: 1.14.0
KFP version: 1.8.22


In [1]:
GOOGLE_CLOUD_PROJECT = 'qwiklabs-gcp-00-e8eee1fc1ae1'      
GOOGLE_CLOUD_REGION = 'us-east1'     
GCS_BUCKET_NAME = GOOGLE_CLOUD_PROJECT + '-gcs'

In [2]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

Updated property [core/project].


In [3]:
PIPELINE_NAME = 'vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

_METADATA_PATH = 'gs://{}/metadata/{}/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME, "metadata.db")

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

PIPELINE_ROOT: gs://qwiklabs-gcp-00-e8eee1fc1ae1-gcs/pipeline_root/vertex-pipelines


In [20]:
!gsutil cp trainer/*.py {MODULE_ROOT}/

Copying file://trainer/constants_file.py [Content-Type=text/x-python]...
Copying file://trainer/trainer_file.py [Content-Type=text/x-python]...          
Copying file://trainer/transform_file.py [Content-Type=text/x-python]...        
- [3 files][  9.8 KiB/  9.8 KiB]                                                
Operation completed over 3 objects/9.8 KiB.                                      


In [21]:
!gsutil cp  ../data/census/data.csv {DATA_ROOT}/

Copying file://../data/census/data.csv [Content-Type=text/csv]...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [22]:
!gsutil cat {DATA_ROOT}/data.csv | head

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-S

## Pipelines

In [4]:
## modunle file 
_trainer_module_file = 'trainer_file.py'
_transform_module_file = 'transform_file.py'

In [5]:
import tensorflow as tf
from tfx import v1 as tfx
import kfp
import os

import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key='label'
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                      threshold=tfma.MetricThreshold(
                          value_threshold=tfma.GenericValueThreshold(
                              lower_bound={'value': 0.5}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}
                                  )
                              )
                      )
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column.
        tfma.SlicingSpec(
            feature_keys=['sex']),
        tfma.SlicingSpec(
            feature_keys=['race']),
    ])


def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     transform_module_file: str, trainer_module_file : str,
                     serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates pipeline with TFX.""" 

  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

  schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])

  example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

  transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(transform_module_file))

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=50),
    eval_args=tfx.proto.EvalArgs(num_steps=50))

  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      statistics_gen,
      schema_gen,
      example_validator,
      transform,
      trainer,
      model_resolver,
      evaluator,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

2023-09-18 03:11:38.687451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libten

## Run the pipeline on Vertex Pipelines

In [6]:
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        trainer_module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        transform_module_file=os.path.join(MODULE_ROOT,_transform_module_file),
        serving_model_dir=SERVING_MODEL_DIR,
        metadata_path=_METADATA_PATH))

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_file.py -> build/lib
copying trainer_file.py -> build/lib
copying constants_file.py -> build/lib


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /var/tmp/tmp_fyqd4cv
running install
running install_lib
copying build/lib/constants_file.py -> /var/tmp/tmp_fyqd4cv
copying build/lib/transform_file.py -> /var/tmp/tmp_fyqd4cv
copying build/lib/trainer_file.py -> /var/tmp/tmp_fyqd4cv
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmp_fyqd4cv/tfx_user_code_Transform-0.0+4dbdc45d93bf36ea5ba3bea6d3806286753d238498752f82aa6c2820663d7a61-py3.10.egg-info
running install_scripts
creating /var/tmp/tmp_fyqd4cv/tfx_user_code_Transform-0.

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


copying build/lib/constants_file.py -> /var/tmp/tmp3jyh536t
copying build/lib/transform_file.py -> /var/tmp/tmp3jyh536t
copying build/lib/trainer_file.py -> /var/tmp/tmp3jyh536t
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmp3jyh536t/tfx_user_code_Trainer-0.0+4dbdc45d93bf36ea5ba3bea6d3806286753d238498752f82aa6c2820663d7a61-py3.10.egg-info
running install_scripts
creating /var/tmp/tmp3jyh536t/tfx_user_code_Trainer-0.0+4dbdc45d93bf36ea5ba3bea6d3806286753d238498752f82aa6c2820663d7a61.dist-info/WHEEL
creating

In [7]:
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/726102867922/locations/us-east1/pipelineJobs/vertex-pipelines-20230918031239
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/726102867922/locations/us-east1/pipelineJobs/vertex-pipelines-20230918031239')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-east1/pipelines/runs/vertex-pipelines-20230918031239?project=726102867922
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/726102867922/locations/us-east1/pipelineJobs/vertex-pipelines-20230918031239 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/726102867922/locations/us-east1/pipelineJobs/vertex-pipelines